In [1]:

import textdistance
import time
import json
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials

In [2]:
'''
Authenticate
Authenticates your credentials and creates a client.
'''


def load_config(config_path='../config.json'):
    with open(config_path, 'r') as config_file:
        config = json.load(config_file)
    return config


config = load_config()
subscription_key = config['subscription_key']
endpoint = config['endpoint']
#
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
'''
END - Authenticate
'''

'\nEND - Authenticate\n'

In [19]:
# img = open("test1.png", "rb")
img = open("images/test2.jpeg", "rb")
read_response = computervision_client.read_in_stream(
    image=img,
    mode="Printed",
    raw=True
)
# print(read_response.as_dict())

operation_id = read_response.headers['Operation-Location'].split('/')[-1]
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status not in ['notStarted', 'running']:
        break
    time.sleep(1)

# Print the detected text, line by line
result = []
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            print(line.text)
            result.append(line.text)

print()

Lucces in resolvarea
TEMELOR la
LABORA toarele de
Inteligenta Artificialà!


In [25]:

# groundTruth = ["Google Cloud", "Platform"]
groundTruth = ["Succes in rezolvarea", "tEMELOR la", "LABORA toarle de", "Inteligenta Artificiala!"]


noOfCorrectLines = sum(i == j for i, j in zip(result, groundTruth))
print(noOfCorrectLines)

0


In [38]:


recognized_lines = ["Lucces in resolvarea", "TEMELOR la", "LABORA toarele de", "Inteligenta Artificialà!"]
ground_truth = ["Succes in rezolvarea", "tEMELOR la", "LABORAtoarele de", "Inteligenta Artificiala!"]

for r_line, g_line in zip(recognized_lines, ground_truth):
    lev_dist = textdistance.levenshtein.distance(r_line, g_line)
    jac_sim = textdistance.jaccard.similarity(r_line, g_line)
    cos_sim = textdistance.cosine.similarity(r_line, g_line)

    print(r_line, "==>", g_line)
    print("\\- Levenshtein Distance:", lev_dist)
    print("\\- Jaccard Similarity:", round(jac_sim, 3))
    print("\\- Cosine Similarity:", round(cos_sim, 3))
    print()

Lucces in resolvarea ==> Succes in rezolvarea
\- Levenshtein Distance: 2
\- Jaccard Similarity: 0.818
\- Cosine Similarity: 0.9

TEMELOR la ==> tEMELOR la
\- Levenshtein Distance: 1
\- Jaccard Similarity: 0.818
\- Cosine Similarity: 0.9

LABORA toarele de ==> LABORAtoarele de
\- Levenshtein Distance: 1
\- Jaccard Similarity: 0.941
\- Cosine Similarity: 0.97

Inteligenta Artificialà! ==> Inteligenta Artificiala!
\- Levenshtein Distance: 1
\- Jaccard Similarity: 0.92
\- Cosine Similarity: 0.958
